# Introduction au Reinforcement Learning

## Le Dilemme Exploration/Exploitation

Dans ce tutoriel, nous allons explorer l'un des concepts fondamentaux du reinforcement learning : le dilemme exploration/exploitation. 

Le **dilemme exploration/exploitation** fait référence au compromis entre :
- **Explorer** de nouvelles actions pour collecter plus d'informations sur l'environnement
- **Exploiter** les connaissances actuelles pour maximiser la récompense immédiate

Nous allons illustrer ce concept à travers le problème classique du **bandit manchot à plusieurs bras** (multi-armed bandit), qui constitue une introduction parfaite au reinforcement learning.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Configuration pour des graphiques plus jolis
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)  # Pour la reproductibilité

## 1. Le problème du bandit manchot à plusieurs bras

Imaginons que nous sommes face à plusieurs machines à sous (bandits manchots). Chaque machine a une probabilité inconnue de donner une récompense. Notre objectif est de maximiser notre gain total sur un nombre fini d'essais.

### Modélisation du problème

- Nous avons `k` machines à sous (bras)
- Chaque machine `i` a une distribution de probabilité inconnue avec une espérance de récompense μ_i
- À chaque tour, nous choisissons une machine et recevons une récompense
- Notre objectif est de maximiser la somme des récompenses sur `T` tours

Commençons par créer une classe pour simuler les machines à sous :

In [ ]:
class BanditEnvironment:
    def __init__(self, k=10, distribution_type='bernoulli'):
        """Initialise un environnement de bandit à k bras
        
        Parameters:
        -----------
        k : int
            Nombre de bras (machines)
        distribution_type : str
            Type de distribution ('bernoulli' ou 'gaussian')
        """
        self.k = k
        self.distribution_type = distribution_type
        
        # Génération des vraies moyennes pour chaque bras
        if distribution_type == 'bernoulli':
            # Pour les distributions de Bernoulli, moyennes entre 0 et 1
            self.true_means = np.random.random(k)
        else:  # 'gaussian'
            # Pour les distributions gaussiennes, moyennes entre 0 et 10
            self.true_means = np.random.normal(5, 2, k)
        
        # Stockage du meilleur bras et de sa moyenne
        self.optimal_arm = np.argmax(self.true_means)
        self.optimal_mean = self.true_means[self.optimal_arm]
    
    def pull(self, arm):
        """Tire le bras spécifié et retourne la récompense
        
        Parameters:
        -----------
        arm : int
            Indice du bras à tirer
            
        Returns:
        --------
        float
            Récompense obtenue
        """
        if arm < 0 or arm >= self.k:
            raise ValueError(f"Bras invalide. Doit être entre 0 et {self.k-1}")
            
        # Génération de la récompense selon la distribution
        if self.distribution_type == 'bernoulli':
            # Pour Bernoulli, 1 avec probabilité true_means[arm], 0 sinon
            return np.random.binomial(1, self.true_means[arm])
        else:  # 'gaussian'
            # Pour Gaussian, centrée sur true_means[arm] avec écart-type 1
            return np.random.normal(self.true_means[arm], 1)
    
    def visualize_arms(self):
        """Visualise les distributions de récompense des bras"""
        plt.figure(figsize=(14, 6))
        
        # Barplot des vraies moyennes
        plt.bar(range(self.k), self.true_means, alpha=0.7)
        plt.axhline(y=self.optimal_mean, color='r', linestyle='--', 
                   label=f'Bras optimal (μ={self.optimal_mean:.3f})')
        
        plt.xlabel('Bras')
        plt.ylabel('Moyenne de récompense (μ)')
        plt.title('Distribution des récompenses moyennes par bras')
        plt.xticks(range(self.k))
        plt.legend()
        plt.show()

Créons un environnement à 10 bras et visualisons les moyennes de récompense des différents bras :

In [ ]:
# Création d'un environnement à 10 bras avec distributions de Bernoulli
env = BanditEnvironment(k=10, distribution_type='bernoulli')
env.visualize_arms()

## 2. Stratégies pour résoudre le dilemme exploration/exploitation

Nous allons maintenant implémenter différentes stratégies pour résoudre le problème du bandit manchot, chacune avec une approche différente du dilemme exploration/exploitation :

1. **Stratégie ε-greedy** : Choisit le meilleur bras connu avec probabilité 1-ε, et explore aléatoirement avec probabilité ε
2. **Stratégie UCB (Upper Confidence Bound)** : Utilise une borne supérieure de confiance pour équilibrer exploration et exploitation
3. **Stratégie Thompson Sampling** : Utilise une approche bayésienne pour modéliser l'incertitude

In [ ]:
class BanditAlgorithm:
    def __init__(self, k, algorithm_type='epsilon-greedy', **kwargs):
        """Initialise un algorithme pour résoudre le problème du bandit
        
        Parameters:
        -----------
        k : int
            Nombre de bras
        algorithm_type : str
            Type d'algorithme ('epsilon-greedy', 'ucb', 'thompson')
        **kwargs : dict
            Paramètres spécifiques à l'algorithme
        """
        self.k = k
        self.algorithm_type = algorithm_type
        
        # Initialisation des estimations de moyenne et du nombre de tirages
        self.q_values = np.zeros(k)  # Estimations des moyennes
        self.n_pulls = np.zeros(k)   # Nombre de tirages par bras
        self.t = 0                   # Nombre total de tirages
        
        # Paramètres spécifiques aux algorithmes
        if algorithm_type == 'epsilon-greedy':
            self.epsilon = kwargs.get('epsilon', 0.1)
        elif algorithm_type == 'ucb':
            self.c = kwargs.get('c', 2.0)  # Paramètre de confiance
        elif algorithm_type == 'thompson':
            # Pour Thompson Sampling avec distribution de Bernoulli
            self.alpha = np.ones(k)  # Succès + 1 (prior)
            self.beta = np.ones(k)   # Échecs + 1 (prior)
    
    def select_arm(self):
        """Sélectionne un bras selon la stratégie de l'algorithme
        
        Returns:
        --------
        int
            Indice du bras choisi
        """
        if np.min(self.n_pulls) == 0:
            # Si certains bras n'ont jamais été tirés, on en choisit un
            return np.where(self.n_pulls == 0)[0][0]
        
        if self.algorithm_type == 'epsilon-greedy':
            # Epsilon-greedy: exploration avec proba epsilon, exploitation sinon
            if np.random.random() < self.epsilon:
                return np.random.randint(self.k)  # Exploration
            else:
                return np.argmax(self.q_values)   # Exploitation
                
        elif self.algorithm_type == 'ucb':
            # UCB: choisit le bras avec la borne supérieure de confiance la plus élevée
            ucb_values = self.q_values + self.c * np.sqrt(np.log(self.t) / self.n_pulls)
            return np.argmax(ucb_values)
            
        elif self.algorithm_type == 'thompson':
            # Thompson Sampling: échantillonne de la distribution a posteriori
            theta_samples = np.random.beta(self.alpha, self.beta)
            return np.argmax(theta_samples)
    
    def update(self, arm, reward):
        """Met à jour les estimations après avoir tiré un bras
        
        Parameters:
        -----------
        arm : int
            Indice du bras tiré
        reward : float
            Récompense obtenue
        """
        # Mise à jour du nombre de tirages
        self.n_pulls[arm] += 1
        self.t += 1
        
        # Mise à jour de l'estimation de la moyenne (moyenne incrémentale)
        self.q_values[arm] += (reward - self.q_values[arm]) / self.n_pulls[arm]
        
        # Mise à jour spécifique pour Thompson Sampling
        if self.algorithm_type == 'thompson':
            if reward == 1:  # Succès
                self.alpha[arm] += 1
            else:  # Échec
                self.beta[arm] += 1

## 3. Simulation et comparaison des stratégies

Nous allons maintenant comparer les performances des différentes stratégies sur notre environnement de bandit.

In [ ]:
def run_experiment(env, algorithm, n_steps=1000):
    """Exécute une expérience avec un algorithme sur un environnement
    
    Parameters:
    -----------
    env : BanditEnvironment
        Environnement à utiliser
    algorithm : BanditAlgorithm
        Algorithme à tester
    n_steps : int
        Nombre d'étapes de simulation
        
    Returns:
    --------
    tuple
        (rewards, optimal_actions)
    """
    rewards = np.zeros(n_steps)
    optimal_actions = np.zeros(n_steps)
    
    for t in range(n_steps):
        # Sélection du bras
        arm = algorithm.select_arm()
        
        # Vérification si l'action est optimale
        optimal_actions[t] = 1 if arm == env.optimal_arm else 0
        
        # Tirage du bras et obtention de la récompense
        reward = env.pull(arm)
        rewards[t] = reward
        
        # Mise à jour de l'algorithme
        algorithm.update(arm, reward)
    
    return rewards, optimal_actions

In [ ]:
# Paramètres de l'expérience
n_steps = 5000
n_runs = 100  # Nombre d'exécutions pour moyenner les résultats
k = 10        # Nombre de bras

# Création des algorithmes à comparer
algorithms = {
    'ε-greedy (ε=0.1)': {'type': 'epsilon-greedy', 'params': {'epsilon': 0.1}},
    'ε-greedy (ε=0.01)': {'type': 'epsilon-greedy', 'params': {'epsilon': 0.01}},
    'UCB (c=2)': {'type': 'ucb', 'params': {'c': 2}},
    'Thompson Sampling': {'type': 'thompson', 'params': {}}
}

# Stockage des résultats
all_rewards = {name: np.zeros((n_runs, n_steps)) for name in algorithms}
all_optimal = {name: np.zeros((n_runs, n_steps)) for name in algorithms}

# Exécution des expériences
for run in tqdm(range(n_runs), desc="Runs"):
    # Création d'un nouvel environnement pour chaque exécution
    env = BanditEnvironment(k=k, distribution_type='bernoulli')
    
    for name, algo_config in algorithms.items():
        # Création de l'algorithme
        algorithm = BanditAlgorithm(
            k=k, 
            algorithm_type=algo_config['type'], 
            **algo_config['params']
        )
        
        # Exécution de l'expérience
        rewards, optimal_actions = run_experiment(env, algorithm, n_steps)
        
        # Stockage des résultats
        all_rewards[name][run] = rewards
        all_optimal[name][run] = optimal_actions

In [ ]:
# Calcul des moyennes et intervalles de confiance
avg_rewards = {name: np.mean(rewards, axis=0) for name, rewards in all_rewards.items()}
avg_optimal = {name: np.mean(optimal, axis=0) for name, optimal in all_optimal.items()}

# Calcul des récompenses cumulatives moyennes
cumulative_rewards = {name: np.cumsum(rewards) / np.arange(1, n_steps + 1) 
                      for name, rewards in avg_rewards.items()}

# Visualisation des résultats
plt.figure(figsize=(15, 10))

# Graphique des récompenses moyennes au fil du temps
plt.subplot(2, 1, 1)
for name, rewards in cumulative_rewards.items():
    plt.plot(rewards, label=name)
plt.xlabel('Étapes')
plt.ylabel('Récompense moyenne')
plt.title('Récompense moyenne au fil du temps')
plt.legend()
plt.grid(True)

# Graphique du pourcentage d'actions optimales au fil du temps
plt.subplot(2, 1, 2)
window = 200  # Fenêtre glissante pour lisser la courbe
for name, optimal in avg_optimal.items():
    # Application d'une moyenne mobile pour lisser la courbe
    smoothed = np.convolve(optimal, np.ones(window)/window, mode='valid')
    plt.plot(np.arange(len(smoothed)), smoothed, label=name)
plt.xlabel('Étapes')
plt.ylabel('% d\'actions optimales')
plt.title('Pourcentage d\'actions optimales au fil du temps')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 4. Analyse du dilemme exploration/exploitation

Nous allons maintenant analyser plus en détail le comportement des algorithmes en termes d'exploration et d'exploitation.

In [ ]:
# Créons un nouvel environnement pour l'analyse détaillée
np.random.seed(123)  # Pour la reproductibilité
env_analysis = BanditEnvironment(k=10, distribution_type='bernoulli')
env_analysis.visualize_arms()

# Exécutons les algorithmes sur cet environnement
algorithms_for_analysis = {
    'ε-greedy (ε=0.1)': BanditAlgorithm(k=10, algorithm_type='epsilon-greedy', epsilon=0.1),
    'UCB (c=2)': BanditAlgorithm(k=10, algorithm_type='ucb', c=2),
    'Thompson Sampling': BanditAlgorithm(k=10, algorithm_type='thompson')
}

# Nombre de tirages par bras pour chaque algorithme
n_pulls_per_arm = {name: np.zeros(10) for name in algorithms_for_analysis}

# Simulation
n_steps_analysis = 2000
for name, algorithm in algorithms_for_analysis.items():
    for _ in range(n_steps_analysis):
        arm = algorithm.select_arm()
        reward = env_analysis.pull(arm)
        algorithm.update(arm, reward)
    
    # Enregistrement du nombre de tirages par bras
    n_pulls_per_arm[name] = algorithm.n_pulls.copy()

In [ ]:
# Visualisation du nombre de tirages par bras
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, n_pulls) in enumerate(n_pulls_per_arm.items()):
    axes[i].bar(range(10), n_pulls, alpha=0.7)
    axes[i].axvline(x=env_analysis.optimal_arm, color='r', linestyle='--', 
                    label=f'Bras optimal')
    axes[i].set_title(f'{name} - Distribution des tirages par bras')
    axes[i].set_xlabel('Bras')
    axes[i].set_ylabel('Nombre de tirages')
    axes[i].set_xticks(range(10))
    axes[i].legend()

plt.tight_layout()
plt.show()

## 5. La valeur de l'information

Une façon de comprendre le dilemme exploration/exploitation est de considérer la **valeur de l'information**. Lorsque nous explorons, nous sacrifions potentiellement une récompense immédiate pour obtenir de l'information qui pourrait mener à de meilleures récompenses futures.

Développons une mesure simple pour évaluer la valeur de l'information pour chaque bras :

In [ ]:
def compute_information_value(q_values, n_pulls, total_steps, exploration_value=0.1):
    """Calcule une mesure simple de la valeur de l'information pour chaque bras
    
    Parameters:
    -----------
    q_values : numpy.ndarray
        Estimations des valeurs des bras
    n_pulls : numpy.ndarray
        Nombre de tirages par bras
    total_steps : int
        Nombre total d'étapes restantes
    exploration_value : float
        Paramètre de pondération pour l'exploration
        
    Returns:
    --------
    numpy.ndarray
        Valeurs d'information pour chaque bras
    """
    # Estimation de l'incertitude (inverse du nombre de tirages)
    uncertainty = 1.0 / np.sqrt(n_pulls + 1)
    
    # Valeur estimée de l'exploitation pure
    exploitation_value = q_values
    
    # Valeur de l'information = valeur d'exploitation + terme d'exploration
    info_value = exploitation_value + exploration_value * uncertainty * np.sqrt(np.log(total_steps))
    
    return info_value

# Créons un nouvel environnement pour l'analyse
np.random.seed(456)
env_info = BanditEnvironment(k=5, distribution_type='bernoulli')
env_info.visualize_arms()

# Initialisons un algorithme ε-greedy
algo_info = BanditAlgorithm(k=5, algorithm_type='epsilon-greedy', epsilon=0.1)

# Simulation pour quelques étapes
n_steps_info = 200
info_values_history = []

for t in range(n_steps_info):
    # Calcul des valeurs d'information avant de prendre la décision
    info_values = compute_information_value(
        algo_info.q_values, 
        algo_info.n_pulls, 
        n_steps_info - t,
        exploration_value=0.1
    )
    info_values_history.append(info_values.copy())
    
    # Sélection du bras et mise à jour
    arm = algo_info.select_arm()
    reward = env_info.pull(arm)
    algo_info.update(arm, reward)

In [ ]:
# Visualisation de l'évolution des valeurs d'information
info_values_history = np.array(info_values_history)

plt.figure(figsize=(12, 6))
for arm in range(5):
    plt.plot(info_values_history[:, arm], label=f'Bras {arm}')

plt.axhline(y=env_info.optimal_mean, color='r', linestyle='--', 
           label=f'Moyenne optimale (μ={env_info.optimal_mean:.3f})')
plt.xlabel('Étapes')
plt.ylabel('Valeur d\'information')
plt.title('Évolution des valeurs d\'information par bras')
plt.legend()
plt.grid(True)
plt.show()